In [2]:
import json
from pathlib import Path
import shutil

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import rioxarray as rxr
from shapely.geometry import Polygon, box
from tqdm import tqdm

from utils.helpers import get_rgb_channels

root_dir = Path('/datadrive/hand_labels')
json_dir =root_dir / 'json_export'
img_input_dir = root_dir / 'import_candidates/input_128px'
img_dir = root_dir / 'images_import'
shp_dir = root_dir / 'shp_labels'

In [6]:

def print_structure(data, indent=0):
    for key, value in data.items():
        print('  ' * indent + str(key))
        if isinstance(value, dict):
            print_structure(value, indent+1)
        elif isinstance(value, list) and isinstance(value[0], dict) and len(value) > 0:
            print_structure(value[0], indent+1)

# for file in json_dir.iterdir():
#     with open(file) as f:
#         data = json.load(f)

#     print_structure(data[0])
#     break


In [3]:
def get_image_names(json_data):
    input_images = []
    for task in json_data:
        img_id = task["file_upload"]
        parts = img_id.split('-', 1) # Split the string at the first occurrence of '-'
        new_string = parts[1] if len(parts) > 1 else parts[0]
        input_images.append(new_string)
    return input_images


def move_files(input_images, img_input_dir, img_dir):
    for img_name in input_images:
        
        tif_name = Path(img_name).with_suffix('.TIF')
        src_tif_file = img_input_dir / "images" / tif_name
        for sub_dir, name in [("png", img_name), ("images", tif_name)]:
            src_file = img_input_dir / sub_dir / name
            if src_file.exists():
                dst_file = img_dir / sub_dir / name
                dst_file.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(src_file, dst_file)
            else:
                print(f'File {src_file} not found.')


def extract_name_from_task(task):
    img_id = task["file_upload"]
    parts = img_id.split('-', 1) # Split the string at the first occurrence of '-'
    return parts[1] if len(parts) > 1 else parts[0]


def get_gdf_from_result(result, img, crs):
    tile_size = int(result["original_width"]) # make sure tiles are square
    points = result["value"]["points"]
    
    points = [img.rio.transform() * (tile_size / 100 * np.array(point))  for point in points]
    # draw.polygon(points, outline ="red")

    polygon = Polygon(points)
    label = result["value"]["polygonlabels"][0]
    gdf = gpd.GeoDataFrame({'image_name': [img.name], 'image_bounds': [box(*img.rio.bounds())], 'geometry': [polygon], 'label': [label]}, geometry='geometry', crs=img.rio.crs)
    gdf = gdf.to_crs(crs)
    return gdf


  
def convert_task_to_gdf(task, crs):
    img_name = extract_name_from_task(task)

    tif_img_dir = Path('/datadrive/hand_labels/images_import/images')
    img_path = (tif_img_dir / img_name).with_suffix('.TIF')
    img = rxr.open_rasterio(img_path, default_name=img_path.name)
    if img.rio.crs != crs:
        img = img.rio.reproject(crs)
        print(f"Reprojecting {img_name}")

    gdf_list = [get_gdf_from_result(result, img, crs) 
                for annotation in task["annotations"] 
                for result in annotation["result"] 
                if result]
    if gdf_list:
        return pd.concat(gdf_list, ignore_index=True)
    else:
        print(f"No labels could be found for task {img_name}.")
    


In [5]:
img_name = '23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r3_c2.png'
r4c2_task = [task for task in data if extract_name_from_task(task) == img_name][0]
tif_img_dir = Path('/datadrive/hand_labels/images_import/images')
img_path = (tif_img_dir / img_name).with_suffix('.TIF')
img = rxr.open_rasterio(img_path, default_name=img_path.name)
if img.rio.crs != crs:
    img = img.rio.reproject(crs)
    print(img)
print(img.rio.bounds())

<xarray.DataArray '23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r3_c2.TIF' (
                                                                                  band: 8,
                                                                                  y: 130,
                                                                                  x: 130)>
array([[[255, 255, 255, ..., 165, 255, 255],
        [255, 255, 255, ..., 165, 255, 255],
        [ 95,  88,  68, ..., 166, 255, 255],
        ...,
        [255, 255,  61, ...,  72,  74,  76],
        [255, 255,  72, ..., 255, 255, 255],
        [255, 255,  88, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 171, 255, 255],
        [255, 255, 255, ..., 172, 255, 255],
        [ 96,  89,  69, ..., 173, 255, 255],
        ...,
        [255, 255,  65, ...,  75,  74,  77],
        [255, 255,  77, ..., 255, 255, 255],
        [255, 255,  94, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 177, 255, 255],
        [255, 255, 255, ..., 1

In [37]:
file_gdfs = []
for file in tqdm(json_dir.iterdir()):
    crs = "EPSG:32616"
    print(f"Processing {file.name}")    
    with open(file) as f:
        data = json.load(f)
        input_images = get_image_names(data)
        print("Fetching images")    
        # move_files(input_images, img_input_dir=img_input_dir, img_dir=img_dir)

        print("Converting to geoDataframe")
        gdf_list = [convert_task_to_gdf(task, crs) for task in data]
        file_gdfs.append(pd.concat(gdf_list, ignore_index=True))
gdf = pd.concat(file_gdfs, ignore_index=True)
# gdf.drop(columns=["image_bounds"]).to_file(shp_dir / "labeled_buildings.shp") # labels geoDF
images_gdf = gdf.set_geometry('image_bounds').groupby(['image_name','image_bounds']).count().reset_index().set_geometry('image_bounds').set_crs(crs)




0it [00:00, ?it/s]

Processing dora-at-2024-01-15-16-42-2d24b852.json
Fetching images
Converting to geoDataframe
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r0_c0.png
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r0_c1.png
No labels could be found for task 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r0_c1.png.
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r0_c2.png
No labels could be found for task 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r0_c2.png.
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r0_c3.png


1it [00:05,  5.09s/it]

Processing frederike-at-2024-02-02-15-56-8f0455d4.json
Fetching images
Converting to geoDataframe


2it [00:13,  6.86s/it]

...

3it [00:23,  8.40s/it]

Processing michael-at-2024-01-29-15-31-50b48d2e.json
Fetching images
Converting to geoDataframe
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r2_c0.png
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r2_c1.png
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r2_c2.png
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r2_c3.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r1_c1.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r1_c2.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r1_c3.png


4it [00:33,  9.15s/it]

Processing nathan-at-2024-02-06-17-54-a0e43c3c.json
Fetching images
Converting to geoDataframe
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r3_c1.png
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r3_c2.png
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r3_c3.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r0_c0.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r0_c1.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r0_c2.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r0_c3.png
Reprojecting 23JUN18160657-M3DS_R4C2-016144386010_01_P001_r5_c1_r3_c0.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r3_c1.png
Reprojecting 23JUN18160713-M3DS_R2C1-016144385010_01_P001_r7_c7_r3_c3.png


5it [00:49,  9.88s/it]


In [38]:
sat_tiles = images_gdf.image_name.apply(lambda x: '-'.join(x.split('-')[:-1]).replace('M3DS', 'PS3DS')).unique()
img_tiles_dir = Path('/datadrive/merged_PS/tiles/images')
regex_match_tiles = []
for prefix in sat_tiles:
    regex_match_tiles.extend(img_tiles_dir.glob(f'{prefix}*'))

boxes_gdfs = []
for img_path in tqdm(regex_match_tiles):
    img = rxr.open_rasterio(img_path, default_name=img_path.name)
    if img.rio.crs != images_gdf.crs:
        img = img.rio.reproject(images_gdf.crs)
    boxes_gdfs.append({'geometry': box(*img.rio.bounds()), 'name': img.name})
tiles_gdf = gpd.GeoDataFrame(boxes_gdfs)
tiles_gdf.crs = images_gdf.crs
# joined_gdf = gpd.sjoin(tiles_gdf, images_gdf, how='inner', op='intersects')
# joined_gdf = gpd.sjoin(images_gdf, tiles_gdf, how='left', op='contains')
# joined_gdf[joined_gdf.name.isna()]
# joined_gdf.shape

  1%|          | 45/4640 [00:00<00:31, 147.07it/s]

100%|██████████| 4640/4640 [01:08<00:00, 67.68it/s] 


In [ ]:
images_gdf2= images_gdf.copy()
images_gdf2['centroid'] = images_gdf2.geometry.centroid
images_gdf2 = images_gdf2.set_geometry('centroid')
tiles_gdf['centroid'] = tiles_gdf.geometry.centroid
tiles_gdf = tiles_gdf.set_geometry('centroid')
joined_gdf = gpd.sjoin_nearest(images_gdf2, tiles_gdf, how='left', distance_col='distance')



In [43]:
model_dir = Path("/datadrive/hand_labels_model")

if not model_dir.exists():
    model_dir.mkdir()
    (model_dir / "images").mkdir()
    (model_dir / "labels").mkdir()

for file_name in joined_gdf[joined_gdf['distance'] == 0].name.unique():
    source_path = img_tiles_dir / file_name
    dest_path = model_dir / "images" / file_name
    shutil.copy2(source_path, dest_path)

gdf.drop(columns=["image_bounds"])[gdf.image_name.isin(joined_gdf[joined_gdf['distance'] == 0].image_name)].to_file(model_dir / "labels/hand_labels.shp")


Bad pipe message: %s [b'\x91\x98\t\xd2\x11U\x91\x1cR\x11\xda\xa2\xef\x00\xbeLR\xc8 \xeamm\xc0\xe7\x8cWw\xba\t]\xee\xbe\xf9\x8b\x8a\xeaGQ\xdaP\xa8]+S\xb2\xe0\x8b\xd5@\xb0\xec\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\xfbXa\xd5\x0c\x95 ;\xea;\x9e.Z\xa1\xf6D\xd6\xb3\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t\x003\x002\x001\x000\xc0\x0e\xc0\x04\x00/\x00\x9a\x00\x99\x00\x98\x00\x97\x00\x96\x00\x07\xc0\x11\xc0\x07\xc0\x0c\xc0\x02\x00\x05\x00\x04\x00\xff\x02']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'.w\xaey\xb9k\xae|\xa21.;-\x00\xd9bk0